In [ ]:
!pip -q uninstall -y numpy torch torchvision torchaudio transformers datasets accelerate evaluate jiwer soundfile
!pip -q install -U numpy==1.26.4
!pip -q install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install -U transformers==4.38.2 datasets accelerate evaluate jiwer soundfile
!apt-get -qq update && apt-get -qq install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 130.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.11.0 requires soundfile>=0.12.1, which is not installed.
fastai 2.8.5 requires torch<2.10,>=1.10, which is not installed.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
torchtune 0.6.1 requires datasets, which is not installed.
peft 0.18.0 requires accelerate>=0.21.0, which is not installed.
peft 0.18.0 requires torch>=1.13.0, which is not installed.
peft 0.18.0 requires transformers, which is not installed.
sentence-transformers 5.1.2 requires torch>=1.11.0, which is not installed.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, which is not installed.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4

In [ ]:
import numpy as np, torch, transformers
print("numpy:", np.__version__)
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)

a = np.array([1,2,3], dtype=np.float32)
t = torch.from_numpy(a)
print("torch-from-numpy OK:", t, t.dtype)


numpy: 1.26.4
torch: 2.5.1+cu121
transformers: 4.38.2
torch-from-numpy OK: tensor([1., 2., 3.]) torch.float32


In [ ]:
from datasets import load_dataset, Audio
import numpy as np
import subprocess, tempfile
import soundfile as sf

ds = load_dataset("sukumbasar/ASR_EchoBase_Raw", split="train")
ds = ds.cast_column("audio", Audio(decode=False))

def load_audio_from_bytes_ffmpeg(example, sr=16000):
    b = example["audio"].get("bytes", None)
    if b is None:
        example["bad_audio"] = True
        example["speech"] = []
        example["sr"] = sr
        return example

    tmp_in = tempfile.NamedTemporaryFile(suffix=".m4a", delete=False)
    tmp_in.write(b); tmp_in.flush()
    in_path = tmp_in.name

    tmp_out = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    out_path = tmp_out.name
    tmp_out.close()

    cmd = ["ffmpeg","-y","-hide_banner","-loglevel","error",
           "-i", in_path, "-ac","1", "-ar", str(sr), out_path]
    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
        speech, read_sr = sf.read(out_path, dtype="float32")
        if speech.ndim > 1:
            speech = speech.mean(axis=1)
        example["bad_audio"] = False
        example["speech"] = speech.astype(np.float32).tolist()  # dataset uyumlu
        example["sr"] = read_sr
        return example
    except subprocess.CalledProcessError as e:
        example["bad_audio"] = True
        example["speech"] = []
        example["sr"] = sr
        example["ffmpeg_err"] = (e.stderr or "")[:300]
        return example

ds2 = ds.map(load_audio_from_bytes_ffmpeg)
ds2 = ds2.filter(lambda x: x["bad_audio"] == False)

print("ds2 rows:", len(ds2))
print("sample speech len:", len(ds2[0]["speech"]), "sr:", ds2[0]["sr"])
print("sample text:", ds2[0]["text"][:80])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/461 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50 [00:00<?, ? examples/s]

ds2 rows: 50
sample speech len: 65179 sr: 16000
sample text: Bugün hava oldukça sakin.


In [ ]:
import torch, re
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

MODEL_ID = "ozcangundes/wav2vec2-large-xlsr-53-turkish"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()
print("device:", device)

def normalize_tr(t: str) -> str:
    t = t.lower()
    t = re.sub(r"[“”\"']", "", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

vocab = processor.tokenizer.get_vocab()
id2token = {i: tok for tok, i in vocab.items()}
blank_id = processor.tokenizer.pad_token_id

def ctc_greedy_rule_decode(logits):
    if logits.ndim == 3:
        logits = logits[0]
    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().tolist()

    collapsed, prev = [], None
    for i in pred_ids:
        if i != prev:
            collapsed.append(i)
        prev = i

    cleaned = [i for i in collapsed if i != blank_id]
    tokens = [id2token.get(i, "") for i in cleaned]
    return normalize_tr("".join(tokens).replace("|", " "))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at ozcangundes/wav2vec2-large-xlsr-53-turkish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ozcangundes/wav2vec2-large-xlsr-53-turkish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

device: cuda


In [ ]:
import numpy as np
import torch

def forward_no_processor(example):
    speech = np.asarray(example["speech"], dtype=np.float32)
    speech = np.nan_to_num(speech)
    input_values = torch.from_numpy(speech).unsqueeze(0).to(device)
    with torch.no_grad():
        return model(input_values).logits

ex = ds2[0]
logits = forward_no_processor(ex)

print("REF :", normalize_tr(ex["text"]))
print("RULE:", ctc_greedy_rule_decode(logits))


REF : bugün hava oldukça sakin.
RULE: bugün hava oldukça sakip


In [ ]:
preds = []
refs = []

for ex in ds2:
    logits = forward_no_processor(ex)
    preds.append(ctc_greedy_rule_decode(logits))
    refs.append(normalize_tr(ex["text"]))

print("done:", len(preds))
print("sample:", preds[0])


done: 50
sample: bugün hava oldukça sakip


In [ ]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

wer = wer_metric.compute(predictions=preds, references=refs)
cer = cer_metric.compute(predictions=preds, references=refs)

print("WER:", wer)
print("CER:", cer)


WER: 0.5675675675675675
CER: 0.13019079685746351


In [ ]:
def score_subset(filter_fn):
    sub_refs, sub_preds = [], []
    for ex in ds2:
        if filter_fn(ex):
            logits = forward_no_processor(ex)
            sub_preds.append(ctc_greedy_rule_decode(logits))
            sub_refs.append(normalize_tr(ex["text"]))
    return (
        wer_metric.compute(predictions=sub_preds, references=sub_refs),
        cer_metric.compute(predictions=sub_preds, references=sub_refs),
        len(sub_refs)
    )

wer_c, cer_c, n_c = score_subset(lambda x: x["is_common"] == True)
wer_p, cer_p, n_p = score_subset(lambda x: x["is_common"] == False)

print("COMMON  n=", n_c, "WER=", wer_c, "CER=", cer_c)
print("PERSON  n=", n_p, "WER=", wer_p, "CER=", cer_p)


COMMON  n= 25 WER= 0.6060606060606061 CER= 0.147008547008547
PERSON  n= 25 WER= 0.5365853658536586 CER= 0.11709913506320692


In [ ]:
from collections import defaultdict

by_spk = defaultdict(list)
for ex in ds2:
    by_spk[str(ex["speaker_id"])].append(ex)

for spk, items in sorted(by_spk.items()):
    sub_refs, sub_preds = [], []
    for ex in items:
        logits = forward_no_processor(ex)
        sub_preds.append(ctc_greedy_rule_decode(logits))
        sub_refs.append(normalize_tr(ex["text"]))
    wer_s = wer_metric.compute(predictions=sub_preds, references=sub_refs)
    cer_s = cer_metric.compute(predictions=sub_preds, references=sub_refs)
    print(f"speaker {spk} | n={len(items)} | WER={wer_s:.4f} | CER={cer_s:.4f}")


speaker 0 | n=10 | WER=0.6806 | CER=0.1476
speaker 1 | n=10 | WER=0.4430 | CER=0.0970
speaker 2 | n=10 | WER=0.4737 | CER=0.0955
speaker 3 | n=10 | WER=0.6471 | CER=0.1537
speaker 4 | n=10 | WER=0.6133 | CER=0.1605


In [ ]:
def pred_hf_decode(example):
    logits = forward_no_processor(example)
    pred_ids = torch.argmax(logits, dim=-1)[0].detach().cpu().numpy()
    text = processor.tokenizer.decode(pred_ids, skip_special_tokens=True)
    return normalize_tr(text)


In [ ]:
preds2, refs2 = [], []
for ex in ds2:
    preds2.append(pred_hf_decode(ex))
    refs2.append(normalize_tr(ex["text"]))

wer2 = wer_metric.compute(predictions=preds2, references=refs2)
cer2 = cer_metric.compute(predictions=preds2, references=refs2)
print("WER:", wer2)
print("CER:", cer2)


WER: 0.5702702702702702
CER: 0.1335578002244669


In [ ]:
import re

def normalize_tr_strict(t: str) -> str:
    t = t.lower()
    # noktalama temizle
    t = re.sub(r"[^a-zçğıöşü0-9\s]", " ", t)
    # çoklu boşluk
    t = re.sub(r"\s+", " ", t).strip()
    return t


In [ ]:
preds3, refs3 = [], []
for ex in ds2:
    preds3.append(normalize_tr_strict(pred_hf_decode(ex)))
    refs3.append(normalize_tr_strict(ex["text"]))

print("WER:", wer_metric.compute(predictions=preds3, references=refs3))
print("CER:", cer_metric.compute(predictions=preds3, references=refs3))


WER: 0.47883597883597884
CER: 0.11429663608562692


In [ ]:
import numpy as np

def trim_silence(x, thr=0.01):
    x = np.asarray(x, dtype=np.float32)
    if len(x) == 0:
        return x
    m = np.max(np.abs(x))
    if m == 0:
        return x
    mask = np.abs(x) > (thr * m)
    if not mask.any():
        return x
    i0 = np.argmax(mask)
    i1 = len(x) - np.argmax(mask[::-1])
    return x[i0:i1]

def forward_no_processor_trim(example):
    speech = trim_silence(example["speech"], thr=0.01)
    speech = np.nan_to_num(np.asarray(speech, dtype=np.float32))
    input_values = torch.from_numpy(speech).unsqueeze(0).to(device)
    with torch.no_grad():
        return model(input_values).logits


In [ ]:
def pred_hf_decode_trim(example):
    logits = forward_no_processor_trim(example)
    pred_ids = torch.argmax(logits, dim=-1)[0].detach().cpu().numpy()
    return normalize_tr(processor.tokenizer.decode(pred_ids, skip_special_tokens=True))

preds4, refs4 = [], []
for ex in ds2:
    preds4.append(normalize_tr_strict(pred_hf_decode_trim(ex)))
    refs4.append(normalize_tr_strict(ex["text"]))

print("WER:", wer_metric.compute(predictions=preds4, references=refs4))
print("CER:", cer_metric.compute(predictions=preds4, references=refs4))


WER: 0.4603174603174603
CER: 0.11085626911314984


In [ ]:
!pip -q install pyctcdecode kenlm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.1/538.1 kB 37.5 MB/s eta 0:00:00


In [ ]:
from pyctcdecode import build_ctcdecoder

labels = [processor.tokenizer.convert_ids_to_tokens(i) for i in range(processor.tokenizer.vocab_size)]
decoder = build_ctcdecoder(labels)  # LM'siz beam bile bazen iyileştirir

def pred_beam(example):
    logits = forward_no_processor_trim(example)[0].detach().cpu().numpy()  # (T,V)
    text = decoder.decode(logits)  # beam search
    return normalize_tr_strict(text)

preds_beam, refs_beam = [], []
for ex in ds2:
    preds_beam.append(pred_beam(ex))
    refs_beam.append(normalize_tr_strict(ex["text"]))

print("WER:", wer_metric.compute(predictions=preds_beam, references=refs_beam))
print("CER:", cer_metric.compute(predictions=preds_beam, references=refs_beam))


WER: 0.4444444444444444
CER: 0.10626911314984709


In [ ]:
import numpy as np

def preprocess_audio(x, trim_thr=0.01):
    x = np.asarray(x, dtype=np.float32)
    x = np.nan_to_num(x)

    # trim
    m = np.max(np.abs(x)) if len(x) else 0
    if m > 0:
        mask = np.abs(x) > (trim_thr * m)
        if mask.any():
            i0 = np.argmax(mask)
            i1 = len(x) - np.argmax(mask[::-1])
            x = x[i0:i1]

    # peak normalize
    m = np.max(np.abs(x)) if len(x) else 0
    if m > 0:
        x = x / m
    return x

def forward_preprocessed(example):
    speech = preprocess_audio(example["speech"], trim_thr=0.008)
    input_values = torch.from_numpy(speech).unsqueeze(0).to(device)
    with torch.no_grad():
        return model(input_values).logits


In [ ]:
def pred_hf_preprocessed(example):
    logits = forward_preprocessed(example)
    pred_ids = torch.argmax(logits, dim=-1)[0].detach().cpu().numpy()
    return normalize_tr_strict(processor.tokenizer.decode(pred_ids, skip_special_tokens=True))

predsP, refsP = [], []
for ex in ds2:
    predsP.append(pred_hf_preprocessed(ex))
    refsP.append(normalize_tr_strict(ex["text"]))

print("WER:", wer_metric.compute(predictions=predsP, references=refsP))
print("CER:", cer_metric.compute(predictions=predsP, references=refsP))


WER: 0.3835978835978836
CER: 0.09212538226299694


In [ ]:
!pip -q install pyctcdecode


In [ ]:
from pyctcdecode import build_ctcdecoder

# tokenizer vocab id -> token list (pyctcdecode bunu ister)
labels = [processor.tokenizer.convert_ids_to_tokens(i) for i in range(processor.tokenizer.vocab_size)]
decoder = build_ctcdecoder(labels)


In [ ]:
def pred_beam_preprocessed(example):
    logits = forward_preprocessed(example)[0].detach().cpu().numpy()  # (T,V)
    text = decoder.decode(logits)  # beam search
    return normalize_tr_strict(text)

predsB, refsB = [], []
for ex in ds2:
    predsB.append(pred_beam_preprocessed(ex))
    refsB.append(normalize_tr_strict(ex["text"]))

print("WER:", wer_metric.compute(predictions=predsB, references=refsB))
print("CER:", cer_metric.compute(predictions=predsB, references=refsB))


WER: 0.37566137566137564
CER: 0.08753822629969418


In [ ]:
def pred_beam_preprocessed_bw(example, bw=100):
    logits = forward_preprocessed(example)[0].detach().cpu().numpy()  # (T,V)
    text = decoder.decode(logits, beam_width=bw)
    return normalize_tr_strict(text)

for bw in [10, 50, 100, 200]:
    preds, refs = [], []
    for ex in ds2:
        preds.append(pred_beam_preprocessed_bw(ex, bw=bw))
        refs.append(normalize_tr_strict(ex["text"]))
    wer = wer_metric.compute(predictions=preds, references=refs)
    cer = cer_metric.compute(predictions=preds, references=refs)
    print(f"beam_width={bw:<3} | WER={wer:.4f} | CER={cer:.4f}")


beam_width=10  | WER=0.4127 | CER=0.0845
beam_width=50  | WER=0.4127 | CER=0.0845
beam_width=100 | WER=0.4127 | CER=0.0845
beam_width=200 | WER=0.4127 | CER=0.0845


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from pyctcdecode import build_ctcdecoder
import torch
import numpy as np

def eval_model(model_id, beam_width=50):
    # 1) model + processor
    proc = Wav2Vec2Processor.from_pretrained(model_id)
    mdl = Wav2Vec2ForCTC.from_pretrained(model_id).to(device).eval()

    # 2) decoder (bu modelin vocab'ına göre)
    labels = [proc.tokenizer.convert_ids_to_tokens(i) for i in range(proc.tokenizer.vocab_size)]
    dec = build_ctcdecoder(labels)

    # 3) predict fn (SENİN EN İYİ preprocess'inle)
    def predict_one(example):
        speech = preprocess_audio(example["speech"], trim_thr=0.008)  # trim + peak normalize
        speech = np.asarray(speech, dtype=np.float32)
        speech = np.nan_to_num(speech)

        input_values = torch.from_numpy(speech).unsqueeze(0).to(device)
        with torch.no_grad():
            logits = mdl(input_values).logits[0].detach().cpu().numpy()  # (T,V)

        text = dec.decode(logits, beam_width=beam_width)
        return normalize_tr_strict(text)

    # 4) eval
    preds, refs = [], []
    for ex in ds2:
        preds.append(predict_one(ex))
        refs.append(normalize_tr_strict(ex["text"]))

    wer = wer_metric.compute(predictions=preds, references=refs)
    cer = cer_metric.compute(predictions=preds, references=refs)

    print(f"\nMODEL: {model_id}")
    print(f"beam_width={beam_width} | WER={wer:.4f} | CER={cer:.4f}")
    return wer, cer


In [ ]:
eval_model("ozcangundes/wav2vec2-large-xlsr-53-turkish", beam_width=50)
eval_model("mpoyraz/wav2vec2-xls-r-300m-cv7-turkish", beam_width=50)
eval_model("m3hrdadfi/wav2vec2-large-xlsr-turkish", beam_width=50)


Some weights of the model checkpoint at ozcangundes/wav2vec2-large-xlsr-53-turkish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ozcangundes/wav2vec2-large-xlsr-53-turkish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho


MODEL: ozcangundes/wav2vec2-large-xlsr-53-turkish
beam_width=50 | WER=0.3757 | CER=0.0875


Some weights of the model checkpoint at mpoyraz/wav2vec2-xls-r-300m-cv7-turkish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at mpoyraz/wav2vec2-xls-r-300m-cv7-turkish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should pr


MODEL: mpoyraz/wav2vec2-xls-r-300m-cv7-turkish
beam_width=50 | WER=0.2037 | CER=0.0413


Some weights of the model checkpoint at m3hrdadfi/wav2vec2-large-xlsr-turkish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at m3hrdadfi/wav2vec2-large-xlsr-turkish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probab


MODEL: m3hrdadfi/wav2vec2-large-xlsr-turkish
beam_width=50 | WER=0.4127 | CER=0.0845


(0.4126984126984127, 0.08448012232415902)

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from pyctcdecode import build_ctcdecoder
import torch

MODEL_ID = "mpoyraz/wav2vec2-xls-r-300m-cv7-turkish"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(device).eval()

labels = [processor.tokenizer.convert_ids_to_tokens(i) for i in range(processor.tokenizer.vocab_size)]
decoder = build_ctcdecoder(labels)

print("Loaded:", MODEL_ID)


Some weights of the model checkpoint at mpoyraz/wav2vec2-xls-r-300m-cv7-turkish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at mpoyraz/wav2vec2-xls-r-300m-cv7-turkish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should pr

Loaded: mpoyraz/wav2vec2-xls-r-300m-cv7-turkish


In [ ]:
import numpy as np

def predict_final(example, beam_width=50):
    speech = preprocess_audio(example["speech"], trim_thr=0.008)
    speech = np.asarray(speech, dtype=np.float32)
    speech = np.nan_to_num(speech)

    input_values = torch.from_numpy(speech).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(input_values).logits[0].detach().cpu().numpy()

    text = decoder.decode(logits, beam_width=beam_width)
    return normalize_tr_strict(text)


In [ ]:
predsF, refsF = [], []
for ex in ds2:
    predsF.append(predict_final(ex, beam_width=50))
    refsF.append(normalize_tr_strict(ex["text"]))

print("FINAL WER:", wer_metric.compute(predictions=predsF, references=refsF))
print("FINAL CER:", cer_metric.compute(predictions=predsF, references=refsF))


FINAL WER: 0.2037037037037037
FINAL CER: 0.04128440366972477
